Qual perfil dos sintomas das pessoas que pegam Covid?

In [ ]:
import pandas as pd
import matplotlib
matplotlib.use('qt5Agg') # case does not matter tkaGg, TkAgg will do
import matplotlib.pyplot as plt
import re
import panel as pn
pn.extension()

import re
import glob


In [2]:
# Gerando os URL dos .csv
caminho = r'dados/dados-sp-'
todos_dados_csv = glob.glob(caminho + "*.csv")

# Colunas a serem lidas
cols = ["sintomas", "evolucaoCaso", "classificacaoFinal"]

# Carregando os dados
dados = pd.concat((pd.read_csv(f, sep=';', encoding='latin1', usecols=cols, infer_datetime_format = False) for f in todos_dados_csv), ignore_index = True)

In [3]:
# Exclui as linhas cujas colunas indicadas abaixo sao null
dados.dropna(subset = ["classificacaoFinal"], inplace = True)
dados.dropna(subset = ["evolucaoCaso"], inplace = True)
# Exclui os dados Cancelados
dados.drop(dados[dados['evolucaoCaso'] == 'Cancelado'].index, inplace = True)

dados.reset_index(drop=True, inplace = True) # Realoca os indices depois das exclusoes

In [4]:
#separação das colunas em variáveis para facilitar a visualizacao
sintomas = dados["sintomas"]
classificacaoFinal = dados["classificacaoFinal"]
evolucaoDoCaso = dados["evolucaoCaso"]

In [5]:
dadosTeste = []
teste4 = ''
string1 = ['Assintomático']
string2 = ['Outros: Paciente assintomático']
string3 = ['Paciente assintomático']


##Alem de separar as palavras para criar uma dados de Sintomas, ainda conta quantos sintomas cada pessoa teve
for i in range(len(sintomas)):
    teste = []
    if(type(sintomas[i]) == str):
        teste.append(re.split(', |,, |,|,,', sintomas[i]))
    else:
        teste.append(teste4)
    
    
    if(teste[0] == string1 or teste[0] == string2 or teste[0] == string3):
        teste[0] = ['Assintomático']
        teste.append(0)
    else:
        if(len(teste[0]) == 10):
            continue
        teste.append(len(teste[0]))
    teste.append(classificacaoFinal[i])
    teste.append(evolucaoDoCaso[i])
    dadosTeste.append(teste)

In [6]:
#criaçao dos dataframes novos que serao utilizados
dadosPanda = pd.DataFrame(dadosTeste)
dadosPandaCovid = pd.DataFrame(dadosTeste)


dadosPanda.rename(columns={0:'sintomas',1:'numSintomasPorPessoas' ,2:'classificacaoFinal', 3:'evolucaoCaso'},inplace = True)
dadosPandaCovid.rename(columns={0:'sintomas',1:'numSintomasPorPessoas' ,2:'classificacaoFinal', 3:'evolucaoCaso'},inplace = True)

In [7]:
#tratamento dos dataframes para retirar os dados desnecessarios
dadosPandaCovid.drop(dadosPandaCovid[dadosPandaCovid['classificacaoFinal'] == 'Descartado'].index, inplace=True)
dadosPandaCovid.drop(dadosPandaCovid[dadosPandaCovid['classificacaoFinal'] == 'Síndrome Gripal Não Especificada'].index, inplace=True)



dadosPanda.reset_index(drop=True, inplace = True)
dadosPandaCovid.reset_index(drop=True, inplace = True)

In [8]:
#criaçao de dataframes com as pessoas que não possuem covid
dadosPandaSemCovidComGripe = dadosPanda.loc[(dadosPanda['classificacaoFinal'] == 'Síndrome Gripal Não Especificada')]
dadosPandaSemCovidSemGripe = dadosPanda.loc[(dadosPanda['classificacaoFinal'] == 'Descartado')]

dadosPandaSemCovidComGripe.reset_index(drop=True, inplace = True)
dadosPandaSemCovidSemGripe.reset_index(drop=True, inplace = True)


In [9]:
sintomasPorPessoaTotal = dadosPanda.numSintomasPorPessoas.value_counts()
sintomasPorPessoaCovid = dadosPandaCovid.numSintomasPorPessoas.value_counts()

Vamos calcular a Probabilidade Condicional de numeros de sintomas e pessoas que pegaram covid

In [10]:
pB = sintomasPorPessoaTotal.sort_index()/sintomasPorPessoaTotal.sum()

P(B) = Probabilidade da pessoa ter "B" sintomas


In [11]:
pA = dadosPandaCovid['classificacaoFinal'].count()/dadosPanda['classificacaoFinal'].count()

P(A) = probabilidade das pessoas com covid confirmado 

In [12]:
pAinterseccaoB = sintomasPorPessoaCovid.sort_index()/sintomasPorPessoaTotal.sum()

P(A intersecçao B)

In [13]:
pBcondicionalA = pAinterseccaoB/pB

#deixa os dados de frequencia para porcentagem (0 a 100%)
for key in pBcondicionalA.keys():
    pBcondicionalA[key] = round(pBcondicionalA[key]*100, 2)

%matplotlib
plt.bar(pBcondicionalA.keys() , pBcondicionalA.values)

plt.xlabel("Número de sintomas")
plt.ylabel("Porcentagem de chance de ser COVID-19")
plt.title('Porcentagem de chance de ser COVID-19 a partir da quantidade de sintomas')


plt.rcParams.update({'font.size': 20})

plt.show()

Using matplotlib backend: Qt5Agg


In [14]:
#cria o boxplot que ajuda na verificação dos dados acima

data = [dadosPandaCovid['numSintomasPorPessoas'], dadosPandaSemCovidSemGripe['numSintomasPorPessoas']]
fig7, ax7 = plt.subplots()
ax7.boxplot(data)
nomes = ['Pessoas com COVID-19' , 'Pessoas sem COVID-19']

plt.xticks([1, 2],nomes)
plt.ylabel("Número de sintomas")
plt.title('BoxPlot do número de sintomas')

Text(0.5, 1.0, 'BoxPlot do número de sintomas')

P(B/A) = Probabilidade da pessoa ter covid a partir da quantidade de sintomas que ela possui

A maior probabilidade da pessoa ter covid é quando ela tem 9 Sintomas, que é 60.83% 

In [15]:
#conta quantas palavras um dataFrame possui e verifica as palavras erradas
def contarPalavras(dados):
    contadorPalavrasCovid = {}
    for linhas in dados['sintomas']:
        for palavra in linhas:
            if(palavra == 'Distúrbios Gustativos'):
                palavra = 'Gustativos'
            elif(palavra == 'Distúrbios Olfativos'):
                palavra = 'Olfativos'
            elif(palavra == 'Dor De Garganta'):
                palavra =  'Dor de Garganta'
            elif(palavra == ''):
                continue
            elif(palavra == 'Dispnéia'):
                palavra = 'Dispneia'
            contadorPalavrasCovid.setdefault(palavra, 0)
            contadorPalavrasCovid[palavra] += 1

    return contadorPalavrasCovid


Chance de desenvolver um sintoma quando tiver covid

Por ser uma distribuição de Bernoulli, a frequencia relativa dada a uma população relativamente grande, também é seu estimador 

In [17]:
contadorPalavrasCovid = contarPalavras(dadosPandaCovid)
sintomasCovid = contadorPalavrasCovid.copy()

#faz a fequencia relativa para cada sintoma com base na quantidade  
for key in contadorPalavrasCovid:
    contadorPalavrasCovid[key] = contadorPalavrasCovid[key]/len(dadosPandaCovid)

Aqui mostra a chance de voce ter um sintoma se voce tem covid

In [19]:
%matplotlib

plt.bar(sorted(contadorPalavrasCovid.keys()), sorted(contadorPalavrasCovid.values()))
plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência Relativa dos sintomas para as pessoas que tiveram COVID-19')

plt.rcParams['xtick.labelsize'] = 12
plt.show()


Using matplotlib backend: Qt5Agg


Agora vamos fazer a mesma análise dos sintomas para obitos e para cura 

In [21]:
contadorPalavrasTotal = contarPalavras(dadosPanda)

sintomasTotal = contadorPalavrasTotal.copy()

In [22]:
dadosPandaCovidObitos = dadosPandaCovid.copy()

dadosPandaCovidObitos.drop(dadosPandaCovidObitos[dadosPandaCovidObitos['evolucaoCaso'] != 'Óbito'].index, inplace=True)
dadosPandaCovidObitos.reset_index(drop=True, inplace = True)

In [23]:
contadorPalavrasCovidObitos = contarPalavras(dadosPandaCovidObitos)

sintomasCovidObitos = contadorPalavrasCovid.copy()

#faz a fequencia relativa para cada sintoma com base na quantidade 
for key in contadorPalavrasCovidObitos:
    contadorPalavrasCovidObitos[key] = contadorPalavrasCovidObitos[key]/sintomasCovid[key]

In [24]:
plt.bar(sorted(contadorPalavrasCovidObitos.keys()), sorted(contadorPalavrasCovidObitos.values()), color='red')

plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência de óbitos de covid para cada sintoma')

plt.show()

In [25]:
dadosPandaCura = dadosPandaCovid.copy()

dadosPandaCura.drop(dadosPandaCura[dadosPandaCura['evolucaoCaso'] != 'Cura'].index, inplace=True)
dadosPandaCura.reset_index(drop=True, inplace = True)

In [26]:
contadorPalavrasCura = contarPalavras(dadosPandaCura)

sintomasCura = contadorPalavrasCura.copy()

#faz a fequencia relativa para cada sintoma com base na quantidade 
for key in contadorPalavrasCura:
    contadorPalavrasCura[key] = contadorPalavrasCura[key]/sintomasCovid[key]

In [28]:
plt.bar(sorted(contadorPalavrasCura.keys()), sorted(contadorPalavrasCura.values()), color='green')

plt.xlabel("sintomas")
plt.ylabel("Frequência Relativa")
plt.title('Frequência de curados para cada sintoma')

plt.show()